## Analisis de la Tasa de Homicidios en EEUU

In [2]:
# importamos librerias
import empiricaldist
import janitor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import sklearn.metrics
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats as ss

### 1. Asignamos una apariencia general a los graficos

In [4]:
# muestra los graficos y los guarda en el Notebooks
%matplotlib inline

sns.set_style(style='whitegrid') #parametros del estilo general de los graficos
sns.set_context(context='notebook') #
plt.rcParams['figure.figsize'] = (11, 9.4)